In [ ]:
!pip install opendatasets tensorflow

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import opendatasets as od

In [ ]:
od.download(
    "<dataset>")

In [ ]:
user_df = pd.read_csv("myanimelist/UserList.csv",usecols=["username"])
anime_df = pd.read_csv("myanimelist/AnimeList.csv",usecols=["anime_id"])

In [ ]:
usernames = tf.constant(ratings_df["username"].astype(str).unique(), dtype=tf.string)
anime_ids = tf.constant(ratings_df["anime_id"].astype(str).unique(), dtype=tf.string)

username_vocabulary = tf.keras.layers.StringLookup(vocabulary=usernames, mask_token=None)
anime_ids_vocabulary = tf.keras.layers.StringLookup(vocabulary=anime_ids, mask_token=None)

In [ ]:
merged_df = pd.merge(anime_df, df, on="anime_id", how="left")

In [ ]:
merged_df.to_csv("anime_images.csv", index=False)

In [ ]:
embedding_dim = 64

user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(len(user_ids_vocabulary.get_vocabulary()), embedding_dim)
])

anime_model = tf.keras.Sequential([
    anime_ids_vocabulary,
    tf.keras.layers.Embedding(len(anime_ids_vocabulary.get_vocabulary()), embedding_dim)
])

In [ ]:
class RecommenderModel(tfrs.Model):
    def __init__(self, user_model, anime_model):
        super().__init__()
        self.user_model = user_model
        self.anime_model = anime_model
        self.task = tfrs.tasks.Retrieval(
        metrics= tfrs.metrics.FactorizedTopK(
            candidates=ratings.batch(128).map(lambda c: (c["anime_id"]))
        ))

    def compute_loss(self, features, training=False):
        user_embeddings = self.user_model(features["user_id"])
        anime_embeddings = self.anime_model(features["anime_id"])
        return self.task(user_embeddings, anime_embeddings)

In [ ]:
model = RecommenderModel(user_model, anime_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

cached_ratings = ratings.shuffle(100_000).batch(8192).cache()
model.fit(cached_ratings, epochs=5)
